## Imports

In [2]:
import cv2
import cupy as cp
import numpy as np

from skimage.metrics import structural_similarity as ssim
from matplotlib import pyplot as plt
from cupyx.scipy.signal import fftconvolve
from cupyx.scipy.spatial.distance import cdist

## Helper Functions

In [68]:
def convolve_arrays(array1: np.ndarray, array2: np.ndarray, mode='same') -> cp.ndarray:
    cupy_array1 = cp.asarray(array1)
    cupy_array2 = cp.asarray(array2)
    return cp.asnumpy(fftconvolve(cupy_array1, cupy_array2, mode=mode))

In [49]:
def gaussian_filter(size, sigma):
    x, y = np.meshgrid(np.linspace(-1, 1, size), np.linspace(-1, 1, size))
    d = np.sqrt(x*x + y*y)
    g = np.exp(-(d**2 / (2.0 * sigma**2)))
    return g / g.sum()

In [5]:
def MSE(arr1, arr2):
    mse = cp.mean((arr1 - arr2) ** 2)
    return mse

In [6]:
def kernel_and_divide(sigma, size, le_input, f_name):
    gaussian = gaussian_filter(size, sigma)
    image_le_temp = convolve_arrays(le_input, gaussian)
    division = cp.divide(cp.asnumpy(le_input), image_le_temp)
    plt.imsave("f_name", cp.asnumpy(division), cmap="gray")

In [7]:
# def sift_similarity(img1_path, img2_path):
#     # Read images
#     img1 = cv2.imread(img1_path, cv2.IMREAD_GRAYSCALE)
#     img2 = cv2.imread(img2_path, cv2.IMREAD_GRAYSCALE)

#     # Initialize SIFT detector
#     sift = cv2.SIFT_create()

#     # Detect and compute SIFT features
#     kp1, des1 = sift.detectAndCompute(img1, None)
#     kp2, des2 = sift.detectAndCompute(img2, None)

#     # Convert descriptors to CuPy arrays for faster computation
#     des1_gpu = cp.asarray(des1)
#     des2_gpu = cp.asarray(des2)

#     # Compute pairwise distances between descriptors
#     distances = cdist(des1_gpu, des2_gpu, metric='euclidean')

#     # Convert distances back to numpy for CPU operations
#     distances_cpu = cp.asnumpy(distances)

#     # Find the best matches
#     bf = cv2.BFMatcher()
#     matches = bf.knnMatch(des1, des2, k=2)

#     # Apply ratio test
#     good_matches = []
#     for m, n in matches:
#         if m.distance < 0.75 * n.distance:
#             good_matches.append(m)

#     # Calculate similarity score
#     similarity_score = len(good_matches) / min(len(kp1), len(kp2))

#     return similarity_score

# # Example usage
# img1_path = 'good.jpg'
# img2_path = 'Prostate.jpg'
# similarity = sift_similarity(img1_path, img2_path)
# print(f"Similarity score: {similarity}")

RuntimeError: pylibraft is not installed

In [66]:
import cv2

def sift_similarity(img1, img2):
    # Initialize SIFT detector
    sift = cv2.SIFT_create()

    # Detect and compute SIFT features
    kp1, des1 = sift.detectAndCompute(img1, None)
    kp2, des2 = sift.detectAndCompute(img2, None)

    # Use brute-force matcher
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(des1, des2, k=2)

    # Apply ratio test
    good_matches = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:
            good_matches.append(m)

    # Calculate similarity score
    similarity_score = len(good_matches) / min(len(kp1), len(kp2))

    return similarity_score

In [63]:
import cv2
import numpy as np
from scipy.spatial.distance import cdist

def sift_similarity_another(img1, img2):
    # Read images

    # Initialize SIFT detector
    sift = cv2.SIFT_create()

    # Detect and compute SIFT features
    kp1, des1 = sift.detectAndCompute(img1, None)
    kp2, des2 = sift.detectAndCompute(img2, None)

    # Compute pairwise distances between descriptors using CPU
    distances = cdist(des1, des2, metric='euclidean')

    # Find the best matches
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(des1, des2, k=2)

    # Apply ratio test
    good_matches = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:
            good_matches.append(m)

    # Calculate similarity score
    similarity_score = len(good_matches) / min(len(kp1), len(kp2))

    return similarity_score

# Example usage
img1_path = 'good.jpg'
img2_path = 'Prostate.jpg'
similarity = sift_similarity_another(img1_path, img2_path)
print(f"Similarity score: {similarity}")


error: OpenCV(4.7.0) :-1: error: (-5:Bad argument) in function 'detectAndCompute'
> Overload resolution failed:
>  - image is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'image'


In [52]:
def uint8_conversion(data):
    info = np.iinfo(data.dtype) # Get the information of the incoming image type
    data = data.astype(np.float64) / info.max # normalize the data to 0 - 1
    data = 255 * data # Now scale by 255
    return data.astype(np.uint8)

## Algo

In [57]:
img1_path = 'good.jpg'
img2_path = 'Prostate.jpg'

target = cv2.imread(img1_path, cv2.IMREAD_GRAYSCALE)
leinput = cv2.imread(img2_path, cv2.IMREAD_GRAYSCALE)

(2048, 2048)

In [39]:
# gauss = gaussian_filter(11, 3)
# convolved = cp.asnumpy(convolve_arrays(leinput, gauss))
# plt.imsave("amgus.jpg", np.divide(convolved, leinput), cmap = "gray")

In [47]:
best_sift = 0
best_sigma = 0
best_kernel = 0

In [70]:
for kernel_size in range(11, 22, 2):
    for sigma in np.arange(0.1, 5, 0.1):
        gauss = gaussian_filter(kernel_size, sigma)
        convolved = cp.asnumpy(convolve_arrays(leinput, gauss))
        division = np.divide(convolved, leinput)

        SIFT = sift_similarity(target, division)

        if SIFT > best_sift:
            best_sift = SIFT
            best_sigma = sigma
            best_kernel = kernel_size

error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\features2d\src\sift.dispatch.cpp:495: error: (-5:Bad argument) image is empty or has incorrect depth (!=CV_8U) in function 'cv::SIFT_Impl::detectAndCompute'


In [72]:
import numpy as np
import cv2
from scipy.signal import convolve2d

def gaussian_filter(size, sigma):
    x, y = np.meshgrid(np.linspace(-1, 1, size), np.linspace(-1, 1, size))
    d = np.sqrt(x*x + y*y)
    g = np.exp(-(d**2 / (2.0 * sigma**2)))
    return g / g.sum()

def convolve_arrays(image, kernel):
    return convolve2d(image, kernel, mode='same', boundary='symm')

def sift_similarity(img1, img2):
    # Convert images to uint8
    img1 = (img1 * 255).astype(np.uint8)
    img2 = (img2 * 255).astype(np.uint8)

    # Initialize SIFT detector
    sift = cv2.SIFT_create()

    # Detect and compute SIFT features
    kp1, des1 = sift.detectAndCompute(img1, None)
    kp2, des2 = sift.detectAndCompute(img2, None)

    # FLANN parameters
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
    search_params = dict(checks=50)

    # FLANN-based matcher
    flann = cv2.FlannBasedMatcher(index_params, search_params)

    matches = flann.knnMatch(des1, des2, k=2)

    # Apply ratio test
    good_matches = []
    for m, n in matches:
        if m.distance < 0.7 * n.distance:
            good_matches.append(m)

    return len(good_matches)

def optimize_gaussian(target, leinput):
    best_sift = 0
    best_sigma = 0
    best_kernel = 0

    for kernel_size in range(11, 22, 2):
        for sigma in np.arange(0.1, 5, 0.1):
            gauss = gaussian_filter(kernel_size, sigma)
            convolved = convolve_arrays(leinput, gauss)
            division = np.divide(convolved, leinput)

            SIFT = sift_similarity(target, division)

            if SIFT > best_sift:
                best_sift = SIFT
                best_sigma = sigma
                best_kernel = kernel_size

    return best_sigma, best_kernel, best_sift

# Example usage
target = cv2.imread('good.jpg', cv2.IMREAD_GRAYSCALE) / 255.0
leinput = cv2.imread('Prostate.jpg', cv2.IMREAD_GRAYSCALE) / 255.0

best_sigma, best_kernel, best_sift = optimize_gaussian(target, leinput)

print(f"Best sigma: {best_sigma}")
print(f"Best kernel size: {best_kernel}")
print(f"Best SIFT similarity: {best_sift}")

Best sigma: 0.30000000000000004
Best kernel size: 15
Best SIFT similarity: 150


In [74]:
leinput = cv2.imread('Prostate.jpg', cv2.IMREAD_GRAYSCALE) / 255.0
gauss = gaussian_filter(15, 0.30000000000000004)
convolved = convolve_arrays(leinput, gauss)
division = np.divide(convolved, leinput)
plt.imsave("what.jpg", division, cmap="gray")